In [1]:
from cv_bridge import CvBridge, CvBridgeError
import cv2
import rospy
import numpy as np
from sensor_msgs.msg import Image

In [16]:



def process_image(color_image, left):
    hsv_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)

    # Define the HSV range for grey color (adjust as needed)
    lower_grey = np.array([0, 0, 50])
    upper_grey = np.array([180, 50, 200])

    # Create a mask for the grey table
    grey_mask = cv2.inRange(hsv_image, lower_grey, upper_grey)

    # Invert the grey mask to get the objects
    objects_mask = cv2.bitwise_not(grey_mask)

    # Create a mask to exclude the fixed base element (adjust as needed)
    exclude_mask = np.ones(objects_mask.shape, dtype=np.uint8) * 255
    height, width = objects_mask.shape[:2]
    
    if left:
        exclude_mask[:30, width-120:] = 0
    else:
        exclude_mask[:80, :130] = 0
        
    # # exclude_mask[:70, :80] = 0  # Left top corner
    # exclude_mask[:30, width-120:] = 0  # Right top corner
    
    # Apply the exclude mask to the objects mask
    objects_mask = cv2.bitwise_and(objects_mask, objects_mask, mask=exclude_mask)

    text = 'left' if left else 'right'
    # Check if there are any objects (non-zero pixels in the objects mask)
    if cv2.countNonZero(objects_mask) > 0:
        print(f"Objects detected on the {text} table!")
        # result_image = color_image.copy()
        # contours, _ = cv2.findContours(objects_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # cv2.drawContours(result_image, contours, -1, (0, 255, 0), 2)
        # cv2.imshow('Objects on Table', result_image)
        # cv2.waitKey(1)  # Add this line to refresh the window
        return True
    else:
        print(f"No objects detected on {text} the table.")
        return False

In [24]:
rospy.init_node("test")
bridge = CvBridge()
left_image = rospy.wait_for_message('/left_camera/color/image_raw',Image)
right_image = rospy.wait_for_message('/right_camera/color/image_raw',Image)
left_color_image = bridge.imgmsg_to_cv2(left_image, "bgr8")
right_color_image = bridge.imgmsg_to_cv2(right_image, "bgr8")

In [25]:
process_image(right_color_image, False)
process_image(left_color_image, True)



Objects detected on the right table!
Objects detected on the left table!


True